# Population Simulator

In [ ]:
import random

In [ ]:
class Individual(object):
    """This class represents an individual."""

    def __init__(self, gamete1, gamete2):
        """
        @param genes1: sequence of alleles on first "chromosome".
        Each allele is a symbol; typically an integer or a random string.
        @param genes2: sequence of alleles on second "chronosome". """
        self.chromosomes = list(zip(gamete1, gamete2))

    @staticmethod
    def create_random(num_genes=1, num_alleles=2):
        gamete1, gamete2 = [[random.randint(0, num_alleles - 1) for _ in range(num_genes)] for g in range(2)]
        return Individual(gamete1, gamete2)

    def gamete(self):
        """Returns random gamete."""
        return [a[random.randint(0, 1)] for a in self.chromosomes]

    def __eq__(self, other):
        """Checks for genetic equality."""
        return self.chromosomes == other.chromosomes

    def __repr__(self):
        return str(self.chromosomes)


In [ ]:
i = Individual(['A', 'B'], ['a', 'b'])
i.gamete()

['a', 'b']

Student Note:

We are adding a function to the population class:


In [ ]:
class Population(object):
    """This class represents the snapshot of a population."""

    def __init__(self, individuals=None):
        self.individuals = individuals or []
        self.females, self.males = None, None

    def __len__(self):
        return len(self.individuals)

    def __iter__(self):
        return iter(self.individuals)

    def add(self, individual):
        self.individuals.append(individual)

    def pick_individual(self):
        """Returns a random individual."""
        return random.choice(self.individuals)

    def random_mating(self):
        """Returns an individual produced via random mating."""
        g1 = self.pick_individual().gamete()
        g2 = self.pick_individual().gamete()
        return Individual(g1, g2)

    def random_hermaphrodite_mating(self):
        """Returns an individual generated via non-self mating,
        aka Banana Slug mating."""
        i1, i2 = random.choices(self.individuals, k=2)
        g1 = i1.gamete()
        g2 = i2.gamete()
        return Individual(g1, g2)

    def partition_genders_randomly(self, prob=0.5):
        """Partition the population in genders, randomly.
        Returns whether there is at least somebody of both genders."""
        self.males, self.females = [], []
        for x in self.individuals:
            if random.random() < prob:
                self.females.append(x)
            else:
                self.males.append(x)
        return len(self.females) > 0 and len(self.males) > 0

    def random_sexual_mating(self, prob=0.5):
        """This is a simple model, in which mating can happen only between
        individuals of opposite sex, and we are given the probability that an
        individual is of a given sex.
        We could do things in another way, but this is ok for now."""
        if self.females is None and self.males is None:
            feasible = self.partition_genders_randomly(prob=prob)
            if not feasible:
                return None
        f = random.choice(self.females)
        m = random.choice(self.males)
        return Individual(f.gamete(), m.gamete())

    def get_generation(self, method, *args, **kwargs):
        """Gets a new generation according to the given method."""
        new_pop = Population()
        for _ in range(len(self.individuals)):
            new_pop.add(method(self, *args, **kwargs))
        return new_pop


    def __repr__(self):
        """You can improve this..."""

        return "\n".join([str(i) for i in self.individuals])


    def calculate_F(self):
        # Calculate F using the correlation between alleles
        num_individuals = len(self.individuals)
        num_genes = len(self.individuals[0].chromosomes)
        observed, expected = 0, 0

        for i in range(num_genes):
            gene_freq = sum(x.chromosomes[i].count(1) for x in self.individuals) / (2 * num_individuals)
            observed += gene_freq * (1 - gene_freq)

            # Hardy-Weinberg equilibrium assumption for expected
            expected += 0.5 * (1 - gene_freq) ** 2 + 0.5 * gene_freq ** 2
        if expected == 0:
          return 1
        else:
          return 1 - observed / expected

    def simulate_inbreeding_increase(self, mating_method, num_generations):
        inbreeding_values = []

        for generation in range(num_generations):
            # Simulate reproduction
            new_generation = self.get_generation(mating_method)

            # Calculate F for the new generation
            f_value = new_generation.calculate_F()

            # Update the population with the new generation
            self.individuals = new_generation.individuals

            # Store F for each generation
            inbreeding_values.append(f_value)

        return inbreeding_values

    def calculate_K(self, mutation_rate):
      num_individuals = len(self.individuals)
      f_value = self.calculate_F()

      if f_value == 0:
        # Handle the case when F is zero to avoid division by zero
          k_value = 1  # You can set it to a default value or handle it differently
      else:
          k_value = 1 / f_value + 4 * num_individuals * mutation_rate

      return k_value

    def simulate_evolution(self, mating_method, mutation_rate, num_generations):
      f_values = []
      k_values = []

      for generation in range(num_generations):
          new_generation = self.get_generation(mating_method)

          # Apply mutation
          for ind in new_generation:
              mutated_chromosomes = []
              for chrom in ind.chromosomes:
                  mutated_allele = [random.randint(0, 1) if random.random() < mutation_rate else allele for allele in chrom]
                  mutated_chromosomes.append(tuple(mutated_allele))
              ind.chromosomes = tuple(mutated_chromosomes)

          f_value = new_generation.calculate_F()
          k_value = new_generation.calculate_K(mutation_rate)

          self.individuals = new_generation.individuals

          f_values.append(f_value)
          k_values.append(k_value)

      return f_values, k_values









In [ ]:
# Example usage:
population = Population([Individual.create_random() for _ in range(100)])

# Simulate inbreeding increase for random mating
random_inbreeding_values = population.simulate_inbreeding_increase(Population.random_mating, num_generations=50)

# Simulate inbreeding increase for hermaphrodite mating
hermaphrodite_inbreeding_values = population.simulate_inbreeding_increase(Population.random_hermaphrodite_mating, num_generations=50)

# Simulate inbreeding increase for sexual mating
sexual_inbreeding_values = population.simulate_inbreeding_increase(Population.random_sexual_mating, num_generations=50)

# Print or analyze the inbreeding values for each mating method
print("Inbreeding Values for Random Mating:", random_inbreeding_values)
print("Inbreeding Values for Hermaphrodite Mating:", hermaphrodite_inbreeding_values)
print("Inbreeding Values for Sexual Mating:", sexual_inbreeding_values)



# random_ratios = [random_inbreeding_values[i] / random_inbreeding_values[i-1] for i in range(1, len(random_inbreeding_values))]

random_ratios = [random_inbreeding_values[i] / random_inbreeding_values[i-1] if random_inbreeding_values[i-1] != 0 else 0 for i in range(1, len(random_inbreeding_values))]

# Calculate the rate of change for hermaphrodite mating
hermaphrodite_ratios = [hermaphrodite_inbreeding_values[i] / hermaphrodite_inbreeding_values[i-1] for i in range(1, len(hermaphrodite_inbreeding_values))]

# Print the ratios for analysis
print("Ratios for Random Mating:", random_ratios)
print("Ratios for Hermaphrodite Mating:", hermaphrodite_ratios)

average_random_ratio = sum(random_ratios) / len(random_ratios)
print("Average Ratio for Random Mating:", average_random_ratio)

average_hermaphrodite_ratio = sum(hermaphrodite_ratios) / len(hermaphrodite_ratios)
print("Average Ratio for Hermaphrodite Mating:", average_hermaphrodite_ratio)

# ------------------------ Part 2 ----------------------------------- #

# Function to simulate random mating
def simulate_random_mating(population, mutation_rate, num_generations):
    return population.simulate_evolution(Population.random_mating, mutation_rate, num_generations)

# Function to simulate banana slug mating
def simulate_banana_slug_mating(population, mutation_rate, num_generations):
    return population.simulate_evolution(Population.random_hermaphrodite_mating, mutation_rate, num_generations)

# Function to simulate sexual mating
def simulate_sexual_mating(population, mutation_rate, num_generations):
    return population.simulate_evolution(Population.random_sexual_mating, mutation_rate, num_generations)

# Example usage
population = Population([Individual.create_random() for _ in range(100)])

# Simulate evolution for random mating
random_f_values, random_k_values = simulate_random_mating(population, mutation_rate=0.01, num_generations=100)

# Simulate evolution for banana slug mating
slug_f_values, slug_k_values = simulate_banana_slug_mating(population, mutation_rate=0.01, num_generations=100)

# Simulate evolution for sexual mating
sexual_f_values, sexual_k_values = simulate_sexual_mating(population, mutation_rate=0.01, num_generations=100)

# Print or analyze the steady-state values of F and K for each mating method
print("Steady-State Values for Random Mating: F =", random_f_values[-1], "K =", random_k_values[-1])
print("Steady-State Values for Banana Slug Mating: F =", slug_f_values[-1], "K =", slug_k_values[-1])
print("Steady-State Values for Sexual Mating: F =", sexual_f_values[-1], "K =", sexual_k_values[-1])


# Validate the relation K = 1 / F for each mating method
random_relation_valid = random_k_values[-1] == 1 / random_f_values[-1]
slug_relation_valid = slug_k_values[-1] == 1 / slug_f_values[-1]
sexual_relation_valid = sexual_k_values[-1] == 1 / sexual_f_values[-1]

print("Validation of K = 1 / F for Random Mating:", random_relation_valid)
print("Validation of K = 1 / F for Banana Slug Mating:", slug_relation_valid)
print("Validation of K = 1 / F for Sexual Mating:", sexual_relation_valid)







Inbreeding Values for Random Mating: [0.00019998000199983323, 0.004987531172069848, 0.0, 0.0031948881789137795, 0.01606983434183118, 0.009752214150661631, 0.012718600953894987, 0.01980198019801982, 0.009752214150661631, 0.0627663696241767, 0.11764705882352944, 0.004987531172069848, 0.004987531172069848, 0.0, 0.007174172977281845, 0.009752214150661631, 0.007174172977281845, 0.01606983434183118, 0.001798381456688869, 0.001798381456688869, 0.01980198019801982, 0.10048437648399644, 0.07692307692307698, 0.24083032808514393, 0.33679833679833693, 0.4256926952141058, 0.4774665042630939, 0.5164305318596545, 0.4, 0.5164305318596545, 0.6196424874042377, 0.5423802929815612, 0.5553308292401042, 0.59402460456942, 0.5034420832086203, 0.5423802929815612, 0.6323851203501093, 0.6450782467312512, 0.7322515212981744, 0.6828240252897786, 0.7923434575206811, 0.7565282268092515, 0.7322515212981744, 0.6577181208053691, 0.5682582862051686, 0.59402460456942, 0.6196424874042377, 0.6828240252897786, 0.68282402528

Here is a simple example

In [ ]:
pop = Population()
for _ in range(10):
    i = Individual.create_random(num_genes=2)
    pop.add(i)
pop

[(0, 1), (1, 0)]
[(0, 0), (0, 1)]
[(0, 1), (0, 1)]
[(1, 0), (1, 1)]
[(1, 1), (0, 0)]
[(1, 1), (1, 1)]
[(1, 0), (0, 0)]
[(1, 0), (0, 0)]
[(1, 0), (1, 0)]
[(0, 0), (1, 1)]

In [ ]:
# Let's simulate 100 generations.
pop = Population()
for _ in range(1000):
    i = Individual.create_random(num_genes=2)
    pop.add(i)
Ft = 0
for _ in range(1000):
    pop = pop.get_generation(Population.random_mating)
pop

[(1, 0), (1, 1)]
[(1, 0), (1, 1)]
[(0, 1), (1, 0)]
[(1, 0), (0, 0)]
[(0, 1), (1, 1)]
[(1, 1), (0, 0)]
[(1, 1), (1, 1)]
[(0, 1), (1, 1)]
[(0, 0), (1, 1)]
[(1, 0), (1, 1)]
[(0, 1), (0, 0)]
[(1, 1), (1, 1)]
[(1, 1), (1, 1)]
[(1, 1), (1, 1)]
[(1, 0), (0, 1)]
[(1, 0), (1, 1)]
[(1, 1), (0, 1)]
[(0, 0), (0, 1)]
[(0, 1), (0, 1)]
[(1, 1), (1, 0)]
[(0, 0), (1, 1)]
[(0, 1), (1, 1)]
[(0, 1), (0, 1)]
[(1, 0), (1, 1)]
[(0, 0), (0, 1)]
[(0, 1), (1, 1)]
[(1, 0), (0, 1)]
[(0, 1), (1, 1)]
[(1, 1), (1, 1)]
[(1, 1), (1, 1)]
[(1, 1), (1, 1)]
[(1, 0), (0, 1)]
[(1, 1), (0, 1)]
[(1, 0), (1, 1)]
[(0, 1), (1, 1)]
[(0, 0), (1, 1)]
[(1, 1), (0, 1)]
[(0, 0), (0, 1)]
[(0, 0), (1, 0)]
[(1, 1), (0, 1)]
[(1, 0), (1, 1)]
[(0, 1), (1, 1)]
[(0, 1), (1, 1)]
[(0, 0), (0, 1)]
[(0, 1), (1, 1)]
[(0, 1), (1, 1)]
[(1, 1), (1, 1)]
[(1, 1), (1, 0)]
[(0, 1), (1, 1)]
[(0, 1), (0, 1)]
[(0, 1), (1, 1)]
[(1, 0), (1, 1)]
[(1, 0), (0, 1)]
[(1, 1), (0, 0)]
[(0, 1), (1, 1)]
[(1, 0), (1, 1)]
[(0, 0), (0, 1)]
[(0, 0), (1, 1)]
[(1, 0), (1, 1

In [ ]:
# code written collectively

